In [1]:
import numpy as np
import cv2
from PIL import Image
# import torch
from xarm.wrapper import XArmAPI  # Import xarm-python-sdk
import genesis as gs
from scipy.spatial.transform import Rotation

SDK_VERSION: 1.15.0


[I 05/13/25 13:55:14.961 283013] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
# Function to convert quaternion to Euler angles
def quatToEuler(q, scalarFirst=True, order='xyz', degrees=False):
    if scalarFirst:
        q = [q[1], q[2], q[3], q[0]]
    r = Rotation.from_quat(q)
    return r.as_euler(order, degrees=degrees)

# Function to get the frame from the camera
def getFrame(cam):
    output = cam.render()
    imageData = output[0]
    frame = cv2.cvtColor(imageData, cv2.COLOR_BGR2RGB)
    return frame

In [3]:
# Initialize Genesis
gs.init(backend=gs.cpu)

scene = gs.Scene(
    show_viewer = True,
    # this is the viewer window that opens while the simulation is running, rather than the camera that records the video
    viewer_options = gs.options.ViewerOptions(
        res           = (1280, 960),
        camera_pos    = (3.5, 0.0, 2.5),
        camera_lookat = (0.0, 0.0, 0.5),
        camera_fov    = 30,
        max_FPS       = 60,
    ),
    vis_options = gs.options.VisOptions(
        show_world_frame = True,
        world_frame_size = 1.0,
        show_link_frame  = False,
        show_cameras     = False,
        plane_reflection = True,
        ambient_light    = (0.1, 0.1, 0.1),
    ),
        sim_options = gs.options.SimOptions(
        dt = 0.01,
        substeps = 4,
    ),
    rigid_options=gs.options.RigidOptions(
        enable_self_collision=True,
    ),
    renderer=gs.renderers.Rasterizer(),
)

plane = scene.add_entity(
    gs.morphs.Plane(),
)
xarm6 = scene.add_entity(
    gs.morphs.URDF(file='../models/ManiSkill-XArm6/mod_xarm6_robotiq.urdf'),

)

box1 = scene.add_entity(
    gs.morphs.Box(
        size=(0.05, 0.05, 0.05),
        pos=(1, 0, 0.0),
    ),
    surface=gs.surfaces.Default(
        color=(1, 0.8, 0),
    )
)


# for long video
camFilm = scene.add_camera(
    res    = (640, 480),
    pos    = (3.5, 0.0, 2.5),
    lookat = (0.65, 0, 0.25),
    fov    = 20,
    GUI    = False,
)

# for vla
cam = scene.add_camera(
    res    = (640, 480),
    pos    = (-2.5, 3, 1.8),
    lookat = (1, 0, 0),
    fov    = 15,
    GUI    = False,
)

scene.build()
camFilm.start_recording()

[Genesis] [13:55:19] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [13:55:19] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [13:55:19] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [13:55:19] [INFO] Running on [11th Gen Intel(R) Core(TM) i9-11900K @ 3.50GHz] with backend gs.cpu. Device memory: 62.64 GB.
[Genesis] [13:55:19] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [13:55:19] [INFO] Scene <59caeb9> created.
[Genesis] [13:55:19] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <1a76ef7>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [13:55:19] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <2d40ab5>, morph: <gs.morphs.URDF(file='/home/a3r/Interns/internship2024-25/models/ManiSkill-XArm6/mod_xarm6_robotiq.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [13:55:19] [INFO] Adding <gs.RigidEntity>. idx: 2, uid: <72b419c>, morph: <

In [4]:
# set up motors and joints
motors_dof = np.arange(10)

jnt_names = [
    'joint1',
    'joint2',
    'joint3',
    'joint4',
    'joint5',
    'joint6',
    'left_outer_knuckle_joint',
    'left_inner_finger_joint',
    'right_outer_knuckle_joint',
    'right_inner_finger_joint'
]
dofs_idx = [xarm6.get_joint(name).dof_idx_local for name in jnt_names]

# pulled from urdf
xarm6.set_dofs_kp(
    np.array([100, 100, 100, 100, 100, 100, 10, 10, 10, 10]),
    motors_dof
)
xarm6.set_dofs_kv(
    np.array([40, 40, 40, 40, 40, 40, 2, 2, 2, 2]),
    motors_dof
)
xarm6.set_dofs_force_range(
    np.array([-6.28318530718, -2.059, -3.8, -6.28318530718, -1.69297, -6.28318530718,  -1, -1, -1, -1]),
    np.array([6.28318530718, 2.0944, 0.19198, 6.28318530718, 3.14159265359, 6.28318530718, 1, 1, 1, 1]),
    motors_dof
)

# get the end-effector link
end_effector = xarm6.get_link('link6')


In [8]:
fingers = (0, 0, 0, 0)

xarm6.control_dofs_position(
    np.array([0, 0, 0, 0, 0, 0, fingers[0], fingers[1], fingers[2], fingers[3]]),
    dofs_idx
)
for i in range (500):
    scene.step()

[Genesis] [13:57:51] [INFO] Running at 0.45 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.47 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.50 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.52 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.55 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.58 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.61 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.64 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.67 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.71 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.75 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.79 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.83 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.87 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.91 FPS.
[Genesis] [13:57:51] [INFO] Running at 0.96 FPS.
[Genesis] [13:57:51] [INFO] Running at 1.01 FPS.
[Genesis] [13:57:51] [INFO] Running at 1.06 FPS.
[Genesis] [13:57:51] [INFO] Running at 1.12 FPS.
[Genesis] [13:57:51] [INFO] Running at 1.18 FPS.
[Genesis] [13:57:51]

In [9]:
fingers = (0.81, -0.88, 0.81, -0.88)

xarm6.control_dofs_position(
    np.array([0, 0, 0, 0, 0, 0, fingers[0], fingers[1], fingers[2], fingers[3]]),
    dofs_idx
)
for i in range (500):
    scene.step()
    

[Genesis] [13:58:15] [INFO] Running at 1.30 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.36 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.44 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.51 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.59 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.67 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.75 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.84 FPS.
[Genesis] [13:58:15] [INFO] Running at 1.94 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.03 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.14 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.25 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.36 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.48 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.60 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.74 FPS.
[Genesis] [13:58:15] [INFO] Running at 2.87 FPS.
[Genesis] [13:58:15] [INFO] Running at 3.02 FPS.
[Genesis] [13:58:15] [INFO] Running at 3.17 FPS.
[Genesis] [13:58:15] [INFO] Running at 3.32 FPS.
[Genesis] [13:58:15]